In [128]:
album

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/1ymIvQpnPQBj1lGlJRqrFQ'},
 'href': 'https://api.spotify.com/v1/albums/1ymIvQpnPQBj1lGlJRqrFQ',
 'id': '1ymIvQpnPQBj1lGlJRqrFQ',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b273b5c6df46431b11d01e890005',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e02b5c6df46431b11d01e890005',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d00004851b5c6df46431b11d01e890005',
   'width': 64}],
 'name': 'Taylor Swift Karaoke',
 'release_date': '2006-10-24',
 'release_date_precision': 'day',
 '

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
taylor_swift_uri = 'spotify:artist:06HL4z0CvFAxyc27GXpf02'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='d519e4f607974394866febb9f7f73805',
                                              client_secret='2c05182d9a9c40dd8e19289c7c6121c0',))

results = spotify.artist_albums(taylor_swift_uri, album_type='album',country='US')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

album_titles=set()
album_id=[]
album_name=set()
for album in albums:
    if 'Karaoke' not in album['name'] and 'Special' not in album['name'] and 'studio' not in album['name'] and 'Live' not in album['name'] and 'Stadium' not in album['name'] and 'Platinum' not in album['name']:
        album_titles.add((album['name'],album['id']))
        album_id.append(album['id'])
        album_name.add(album['name'])

comp_list = []

for album_name, album_id in album_titles:
    tracks = spotify.album_tracks(album_id)['items']
    for track in tracks:
        track_name = track['name']
        track_id = track['id']
        comp_list.append((album_name,album_id,track_name,track_id))

len(comp_list)

447

In [8]:
import pandas as pd

In [ ]:
features = spotify.audio_features(tracks=['4g2c7NoTWAOSYDy44l9nub'])

In [11]:
pd.Series(features[0])

danceability                                                    0.735
energy                                                          0.444
key                                                                10
loudness                                                      -10.519
mode                                                                1
speechiness                                                    0.0684
acousticness                                                    0.204
instrumentalness                                               0.0012
liveness                                                         0.17
valence                                                        0.0984
tempo                                                          97.038
type                                                   audio_features
id                                             4g2c7NoTWAOSYDy44l9nub
uri                              spotify:track:4g2c7NoTWAOSYDy44l9nub
track_href          

In [14]:
df = pd.DataFrame()

for album_name,album_id,track_name, track_id in comp_list:
    features = spotify.audio_features(tracks=[track_id])[0]
    features['album_name'] = album_name
    features['album_id'] = album_id
    features['track_name'] = track_name
    ser = pd.Series(features)
    df = pd.concat([df,ser],axis=1)

In [15]:
df = df.transpose().set_index('id')
df.head()[['danceability', 'energy', 'key', 'acousticness','album_name','track_name']]

,danceability,energy,key,acousticness,album_name,track_name
id,,,,,,
4g2c7NoTWAOSYDy44l9nub,0.735,0.444,10,0.204,Midnights (3am Edition),Lavender Haze
199E1RRrVmVTQqBXih5qRC,0.658,0.378,7,0.0593,Midnights (3am Edition),Maroon
02Zkkf2zMkwRGQjZ7T4p8f,0.638,0.634,4,0.133,Midnights (3am Edition),Anti-Hero
6ADDIJxxqzM9LMpm78yzQG,0.659,0.323,9,0.735,Midnights (3am Edition),Snow On The Beach (feat. Lana Del Rey)
7gVWKBcfIW93YxNBi3ApIE,0.694,0.38,2,0.416,Midnights (3am Edition),"You're On Your Own, Kid"
